<a href="https://colab.research.google.com/github/Bikmul/Bikmul/blob/main/fashion_mnist_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оптимизация гиперпараметров нейросети с помощью [Keras Tuner](https://github.com/keras-team/keras-tuner)

Учебный курс "[Программирование нейросетей на Python](https://www.asozykin.ru/courses/nnpython)".

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [2]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.3 MB/s eta 0:00:00


## Подключаем нужные пакеты

In [3]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


<ipython-input-3-b7a017acafd2>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization


## Подготовка данных для обучения сети

In [4]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255
x_test = x_test / 255
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [10]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    # Добавляем случайное количество скрытых слоев
    for i in range(hp.Int('num_hidden_layers', 1, 4)):  # Гиперпараметр num_hidden_layers от 1 до 3
        model.add(Dense(units=hp.Int(f'units_hidden_{i}',
                                   min_value=128,
                                   max_value=600,
                                   step=32),
                    activation=activation_choice))
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров:
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [11]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать -
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения
    directory='test_directory'   # каталог, куда сохраняются обученные сети
    )

## Запускаем подбор гиперпараметров

Пространство поиска

In [12]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
num_hidden_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
units_hidden_0 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


Подбор гиперпараметров

In [13]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Trial 80 Complete [00h 00m 42s]
val_accuracy: 0.8503333330154419

Best val_accuracy So Far: 0.8987500071525574
Total elapsed time: 00h 45m 06s


## Выбираем лучшую модель

In [14]:
tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 37 summary
Hyperparameters:
activation: relu
units_input: 768
num_hidden_layers: 2
units_hidden_0: 384
optimizer: adam
units_hidden_1: 512
units_hidden_2: 256
units_hidden_3: 544
Score: 0.8987500071525574

Trial 14 summary
Hyperparameters:
activation: tanh
units_input: 640
num_hidden_layers: 2
units_hidden_0: 512
optimizer: adam
units_hidden_1: 160
units_hidden_2: 512
units_hidden_3: 192
Score: 0.8981666564941406

Trial 30 summary
Hyperparameters:
activation: relu
units_input: 896
num_hidden_layers: 3
units_hidden_0: 320
optimizer: adam
units_hidden_1: 352
units_hidden_2: 320
units_hidden_3: 384
Score: 0.8979166746139526

Trial 09 summary
Hyperparameters:
activation: relu
units_input: 832
num_hidden_layers: 1
units_hidden_0: 320
optimizer: adam
units_hidden_1: 512
units_hidden_2: 448
units_hidden_3: 320
Score: 0.8975833058357239

Trial 34 summary
Hype

Получаем три лучших модели

In [17]:
models = tuner.get_best_models(num_models=4)

Оцениваем качество модели на тестовых данных

In [18]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 768)               602880    
                                                                 
 dense_1 (Dense)             (None, 384)               295296    
                                                                 
 dense_2 (Dense)             (None, 512)               197120    
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 1100426 (4.20 MB)
Trainable params: 1100426 (4.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.3702 - accuracy: 0.8938

Model: "sequential"
________________________________________